In [ ]:
import Pkg;
Pkg.add("JuMP")
Pkg.add("Ipopt")

In [15]:
# 1. Import Packages
using JuMP, Ipopt

# 2. Model Setup
model = Model(Ipopt.Optimizer)

# 3. Parameters
alpha = 1 # A's marginal rate of substitution between consumption and leisure
beta = 1 # the weight the social planner places on individual B's utility relative to individual A
gamma = 1 # productivity differential between firms X and Y

# 4. Variables
@variable(model, k_A >= 0) # Consumption for individual A
@variable(model, k_B >= 0) # Consumption for individual B
@variable(model, l_A >= 0) # Leisure for individual A
@variable(model, l_B >= 0) # Leisure for individual B
@variable(model, h_X >= 0) # labor input for individual A
@variable(model, h_Y >= 0) # labor input for individual B


# Utility Functions
utility_A(k_A, l_A) = (alpha * k_A) + l_A
utility_B(k_B, l_B) = log(k_B) + log(l_B)

# Production Functions
production_X(h_X) = sqrt(gamma * h_X)
production_Y(h_Y) = sqrt(h_Y)

# 5. Constraints
#@constraint(model, l_A + k_A <= 1)
#@constraint(model, l_B + k_B <= 1)

@constraint(model, l_A + l_B + h_X + h_Y <= 2) # Time resource constraint
@constraint(model, l_A + l_B <= 2)  # Time constraint: Total leisure cannot exceed the time available. Not a necessary constraint
@constraint(model, production_X(h_X) + production_Y(h_Y) == k_A + k_B) # Consumption good resource constraint
 
# 6. Objective Function (Social Planner's Problem)
@objective(model, Max, utility_A(k_A, l_A) + beta*(utility_B(k_B, l_B)))

# 7. Solver Configuration and Solution
optimize!(model)

# 8. Output
println("S = u_A + beta * u_B = ", objective_value(model))
println()
println("k_A: ", value(k_A), ", k_B: ", value(k_B))
println("l_A: ", value(l_A), ", l_B: ", value(l_B))
println("h_X: ", value(h_X), " h_Y: ", value(h_Y))
println()
println("u_A(k_A, l_A) = ", utility_A(value(k_A), value(l_A)))
println("u_B(k_B, l_B) = ", utility_B(value(k_B), value(l_B)))
println("h = 2 - l_A - l_B = ", 2 - value(l_A) - value(l_B))
println("k = F_X(h_X) + F_Y(h_Y) = ", production_X(value(h_X)) + production_Y(value(h_Y)))

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -9.1903424e+00 1.80e-01 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 